# Bias-Variance Tradeoff - Vereinfachte Demonstration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_08/bias_variance_simple.ipynb)

Diese Demonstration zeigt den Bias-Variance Tradeoff anhand einer schwach gekrümmten quadratischen Funktion.

## Konfigurierbare Parameter
- **Quadratische Krümmung**: Stärke der Nichtlinearität  
- **Anzahl Datenpunkte**: Für jede Stichprobe
- **Anzahl Stichproben**: Für Varianz-Berechnung
- **Polynomgrad**: Komplexität des flexiblen Modells
- **Rauschen**: Stärke des Zufallsfehlers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# =============================================================================
# KONFIGURIERBARE PARAMETER
# =============================================================================

# Daten-Parameter
N_POINTS = 30           # Anzahl Datenpunkte pro Stichprobe
N_SAMPLES = 100         # Anzahl Stichproben für Bias-Variance Analyse
NOISE_STD = 0.3         # Standardabweichung des Rauschens
X_RANGE = (-2, 2)       # Bereich der x-Werte

# Wahre Funktion (schwach gekrümmte Parabel)
QUADRATIC_STRENGTH = 0.3  # Stärke der quadratischen Krümmung (0 = linear, 1 = stark gekrümmt)
LINEAR_SLOPE = 1.0        # Linearer Anteil
INTERCEPT = 0.5           # y-Achsenabschnitt

# Modell-Parameter
POLYNOMIAL_DEGREE = 8     # Grad des flexiblen Polynommodells (erhöht für mehr Varianz!)

def true_function(x):
    """Wahre Funktion: schwach gekrümmte Parabel"""
    return INTERCEPT + LINEAR_SLOPE * x + QUADRATIC_STRENGTH * x**2

print(f"Wahre Funktion: f(x) = {INTERCEPT} + {LINEAR_SLOPE}*x + {QUADRATIC_STRENGTH}*x²")
print(f"Datenpunkte pro Stichprobe: {N_POINTS}")
print(f"Anzahl Stichproben: {N_SAMPLES}")
print(f"Rauschen (σ): {NOISE_STD}")
print(f"Polynomgrad (flexibles Modell): {POLYNOMIAL_DEGREE}")

In [ ]:
# Test-x-Werte für Vorhersagen
x_test = np.linspace(X_RANGE[0], X_RANGE[1], 100)
y_true_test = true_function(x_test)

# Speicher für Vorhersagen
predictions_linear = []
predictions_poly = []

# Speicher für eine Beispiel-Stichprobe (für Punktwolke)
sample_x = None
sample_y = None

# Generiere N_SAMPLES verschiedene Datensätze und trainiere Modelle
np.random.seed(42)  # Für Reproduzierbarkeit

for i in range(N_SAMPLES):
    # Generiere Stichprobe
    x_sample = np.random.uniform(X_RANGE[0], X_RANGE[1], N_POINTS)
    y_true_sample = true_function(x_sample)
    noise = np.random.normal(0, NOISE_STD, N_POINTS)
    y_sample = y_true_sample + noise
    
    # Speichere erste Stichprobe für Visualisierung
    if i == 0:
        sample_x = x_sample.copy()
        sample_y = y_sample.copy()
    
    # Trainiere lineares Modell
    model_linear = LinearRegression()
    model_linear.fit(x_sample.reshape(-1, 1), y_sample)
    pred_linear = model_linear.predict(x_test.reshape(-1, 1))
    predictions_linear.append(pred_linear)
    
    # Trainiere Polynommodell
    model_poly = Pipeline([
        ('poly', PolynomialFeatures(degree=POLYNOMIAL_DEGREE)),
        ('linear', LinearRegression())
    ])
    model_poly.fit(x_sample.reshape(-1, 1), y_sample)
    pred_poly = model_poly.predict(x_test.reshape(-1, 1))
    predictions_poly.append(pred_poly)

# Konvertiere zu numpy arrays
predictions_linear = np.array(predictions_linear)
predictions_poly = np.array(predictions_poly)

print(f"✓ {N_SAMPLES} Modelle trainiert")
print(f"  - Lineare Modelle: {predictions_linear.shape}")
print(f"  - Polynomial Modelle (Grad {POLYNOMIAL_DEGREE}): {predictions_poly.shape}")
print(f"  - Beispiel-Stichprobe gespeichert: {len(sample_x)} Punkte")

In [ ]:
# =============================================================================
# BIAS-VARIANCE BERECHNUNG
# =============================================================================

def calculate_bias_variance_mse(predictions, y_true):
    """Berechnet Bias, Variance und MSE"""
    # Mittlere Vorhersage über alle Modelle
    mean_prediction = np.mean(predictions, axis=0)
    
    # Bias² = (Mittelwert der Vorhersagen - wahre Werte)²
    bias_squared = np.mean((mean_prediction - y_true)**2)
    
    # Varianz = Erwartungswert der quadrierten Abweichungen vom Mittelwert
    variance = np.mean(np.var(predictions, axis=0))
    
    # MSE = Bias² + Varianz + Rauschen²
    noise_squared = NOISE_STD**2
    mse_theoretical = bias_squared + variance + noise_squared
    
    return bias_squared, variance, noise_squared, mse_theoretical, mean_prediction

# Berechne Metriken für beide Modelltypen
bias2_linear, var_linear, noise2, mse_linear, mean_pred_linear = calculate_bias_variance_mse(predictions_linear, y_true_test)
bias2_poly, var_poly, _, mse_poly, mean_pred_poly = calculate_bias_variance_mse(predictions_poly, y_true_test)

# Ergebnisse ausgeben
print("=" * 60)
print("BIAS-VARIANCE ANALYSE")
print("=" * 60)
print(f"{'Metrik':<15} {'Linear':<12} {'Polynom':<12} {'Differenz':<12}")
print("-" * 60)
print(f"{'Bias²':<15} {bias2_linear:<12.4f} {bias2_poly:<12.4f} {bias2_linear-bias2_poly:<12.4f}")
print(f"{'Varianz':<15} {var_linear:<12.4f} {var_poly:<12.4f} {var_linear-var_poly:<12.4f}")
print(f"{'Rauschen²':<15} {noise2:<12.4f} {noise2:<12.4f} {0:<12.4f}")
print(f"{'MSE (gesamt)':<15} {mse_linear:<12.4f} {mse_poly:<12.4f} {mse_linear-mse_poly:<12.4f}")
print("-" * 60)
print(f"\n🎯 **Bias-Variance Tradeoff erkennbar:**")
print(f"   • Lineares Modell: Hoher Bias ({bias2_linear:.3f}), niedrige Varianz ({var_linear:.3f})")
print(f"   • Polynom Modell: Niedriger Bias ({bias2_poly:.3f}), höhere Varianz ({var_poly:.3f})")

In [ ]:
# =============================================================================
# VISUALISIERUNG
# =============================================================================

fig = plt.figure(figsize=(18, 12))

# Erstelle ein 2x3 Layout für 6 Subplots
ax1 = plt.subplot(2, 3, 1)
ax2 = plt.subplot(2, 3, 2)
ax3 = plt.subplot(2, 3, 3)
ax4 = plt.subplot(2, 3, 4)
ax5 = plt.subplot(2, 3, 5)
ax6 = plt.subplot(2, 3, 6)

# 1. Beispiel-Datenpunkte mit wahrer Funktion
ax1.scatter(sample_x, sample_y, c='gray', alpha=0.6, s=50, label='Datenpunkte (mit Rauschen)')
ax1.plot(x_test, y_true_test, 'k-', linewidth=3, label='Wahre Funktion', alpha=0.8)
ax1.set_title('Beispiel-Stichprobe', fontsize=14, fontweight='bold')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Wahre Funktion und mittlere Vorhersagen
ax2.plot(x_test, y_true_test, 'k-', linewidth=3, label='Wahre Funktion', alpha=0.8)
ax2.plot(x_test, mean_pred_linear, 'r--', linewidth=2, label=f'Mittlere Vorhersage (Linear)')
ax2.plot(x_test, mean_pred_poly, 'b--', linewidth=2, label=f'Mittlere Vorhersage (Polynom Grad {POLYNOMIAL_DEGREE})')
ax2.set_title('Wahre Funktion vs. Mittlere Vorhersagen', fontsize=14, fontweight='bold')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Alle linearen Modelle
for i in range(min(20, N_SAMPLES)):  # Zeige max. 20 Modelle
    ax3.plot(x_test, predictions_linear[i], 'r-', alpha=0.2, linewidth=1)
ax3.plot(x_test, y_true_test, 'k-', linewidth=3, label='Wahre Funktion')
ax3.plot(x_test, mean_pred_linear, 'r-', linewidth=3, label='Mittlere Vorhersage')
ax3.set_title(f'Lineare Modelle (erste {min(20, N_SAMPLES)} von {N_SAMPLES})', fontsize=14, fontweight='bold')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Alle Polynommodelle
for i in range(min(20, N_SAMPLES)):  # Zeige max. 20 Modelle
    ax4.plot(x_test, predictions_poly[i], 'b-', alpha=0.2, linewidth=1)
ax4.plot(x_test, y_true_test, 'k-', linewidth=3, label='Wahre Funktion')
ax4.plot(x_test, mean_pred_poly, 'b-', linewidth=3, label='Mittlere Vorhersage')
ax4.set_title(f'Polynom Modelle Grad {POLYNOMIAL_DEGREE} (erste {min(20, N_SAMPLES)} von {N_SAMPLES})', fontsize=14, fontweight='bold')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Bias-Variance Balkendiagramm
models = ['Linear', f'Polynom\\nGrad {POLYNOMIAL_DEGREE}']
bias_values = [bias2_linear, bias2_poly]
var_values = [var_linear, var_poly]
noise_values = [noise2, noise2]

x_pos = np.arange(len(models))
width = 0.6

# Gestapeltes Balkendiagramm
p1 = ax5.bar(x_pos, bias_values, width, label='Bias²', color='lightcoral')
p2 = ax5.bar(x_pos, var_values, width, bottom=bias_values, label='Varianz', color='lightblue')
p3 = ax5.bar(x_pos, noise_values, width, bottom=np.array(bias_values)+np.array(var_values), 
             label='Rauschen²', color='lightgray')

ax5.set_title('Bias-Variance Decomposition', fontsize=14, fontweight='bold')
ax5.set_xlabel('Modelltyp')
ax5.set_ylabel('MSE Komponenten')
ax5.set_xticks(x_pos)
ax5.set_xticklabels(models)
ax5.legend()
ax5.grid(True, alpha=0.3, axis='y')

# Werte in die Balken schreiben
for i, (bias, var, noise) in enumerate(zip(bias_values, var_values, noise_values)):
    ax5.text(i, bias/2, f'{bias:.3f}', ha='center', va='center', fontweight='bold')
    ax5.text(i, bias + var/2, f'{var:.3f}', ha='center', va='center', fontweight='bold')
    ax5.text(i, bias + var + noise/2, f'{noise:.3f}', ha='center', va='center', fontweight='bold')

# 6. Varianz-Vergleich als Linienplot
variance_linear_per_point = np.var(predictions_linear, axis=0)
variance_poly_per_point = np.var(predictions_poly, axis=0)

ax6.plot(x_test, variance_linear_per_point, 'r-', linewidth=2, label='Varianz Linear')
ax6.plot(x_test, variance_poly_per_point, 'b-', linewidth=2, label=f'Varianz Polynom Grad {POLYNOMIAL_DEGREE}')
ax6.set_title('Varianz entlang x-Achse', fontsize=14, fontweight='bold')
ax6.set_xlabel('x')
ax6.set_ylabel('Varianz')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n📊 **Interpretation der erweiterten Visualisierung:**")
print(f"   • Panel 1: Beispiel-Datenpunkte mit Rauschen")
print(f"   • Panel 2: Mittlere Vorhersagen zeigen systematische Abweichungen (Bias)")
print(f"   • Panel 3+4: Streuung der Modelle - Polynome Grad {POLYNOMIAL_DEGREE} zeigen deutlich höhere Varianz!")
print(f"   • Panel 5: Bias-Variance Tradeoff - Linear: hoher Bias, Polynom: hohe Varianz")
print(f"   • Panel 6: Varianz entlang x - Polynome zeigen besonders an den Rändern extreme Varianz")
print(f"   • ✅ Klarer Tradeoff: Linear (Bias={bias2_linear:.3f}) vs. Polynom (Varianz={var_poly:.3f})")

In [ ]:
# =============================================================================
# EINZELPLOT-BEISPIELE: 5x GERADE + 5x POLYNOM
# =============================================================================

fig, axes = plt.subplots(2, 5, figsize=(20, 8))

# Generiere 5 verschiedene Stichproben für Einzelplots
np.random.seed(123)  # Andere Seed für Abwechslung

# Obere Reihe: 5 Lineare Modelle
for i in range(5):
    ax = axes[0, i]
    
    # Generiere neue Stichprobe
    x_sample = np.random.uniform(X_RANGE[0], X_RANGE[1], N_POINTS)
    y_true_sample = true_function(x_sample)
    noise = np.random.normal(0, NOISE_STD, N_POINTS)
    y_sample = y_true_sample + noise
    
    # Trainiere lineares Modell
    model_linear = LinearRegression()
    model_linear.fit(x_sample.reshape(-1, 1), y_sample)
    pred_linear = model_linear.predict(x_test.reshape(-1, 1))
    
    # Plot
    ax.scatter(x_sample, y_sample, c='gray', alpha=0.7, s=40, label='Datenpunkte')
    ax.plot(x_test, y_true_test, 'k-', linewidth=2, label='Wahre Funktion')
    ax.plot(x_test, pred_linear, 'r--', linewidth=2, label='Lineares Modell')
    ax.set_title(f'Linear #{i+1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('x')
    if i == 0:
        ax.set_ylabel('y')
        ax.legend(loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-1, 4)  # Einheitliche y-Achse

# Untere Reihe: 5 Polynommodelle
for i in range(5):
    ax = axes[1, i]
    
    # Generiere neue Stichprobe (gleicher Seed-Offset für Vergleichbarkeit)
    np.random.seed(123 + i)
    x_sample = np.random.uniform(X_RANGE[0], X_RANGE[1], N_POINTS)
    y_true_sample = true_function(x_sample)
    noise = np.random.normal(0, NOISE_STD, N_POINTS)
    y_sample = y_true_sample + noise
    
    # Trainiere Polynommodell
    model_poly = Pipeline([
        ('poly', PolynomialFeatures(degree=POLYNOMIAL_DEGREE)),
        ('linear', LinearRegression())
    ])
    model_poly.fit(x_sample.reshape(-1, 1), y_sample)
    pred_poly = model_poly.predict(x_test.reshape(-1, 1))
    
    # Plot
    ax.scatter(x_sample, y_sample, c='gray', alpha=0.7, s=40, label='Datenpunkte')
    ax.plot(x_test, y_true_test, 'k-', linewidth=2, label='Wahre Funktion')
    ax.plot(x_test, pred_poly, 'b--', linewidth=2, label=f'Polynom Grad {POLYNOMIAL_DEGREE}')
    ax.set_title(f'Polynom #{i+1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('x')
    if i == 0:
        ax.set_ylabel('y')
        ax.legend(loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-1, 4)  # Einheitliche y-Achse

plt.suptitle('Einzelmodell-Beispiele: Bias vs. Varianz', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("🔍 **Beobachtungen aus den Einzelplots:**")
print(f"   • **Lineare Modelle (obere Reihe):** Konsistent, aber systematisch falsch (hoher Bias)")
print(f"   • **Polynom Modelle Grad {POLYNOMIAL_DEGREE} (untere Reihe):** Sehr unterschiedlich je nach Datenpunkten (hohe Varianz)")
print(f"   • **Bias:** Lineare Modelle können die Krümmung nicht erfassen → systematische Abweichung")
print(f"   • **Varianz:** Polynome reagieren stark auf zufällige Datenpunkt-Positionen → instabile Vorhersagen")
print(f"   • **Tradeoff:** Einfache Modelle (stabil aber ungenau) vs. Komplexe Modelle (genau aber instabil)")